<a href="https://colab.research.google.com/github/BingHungLee/Heat_Transfer/blob/master/%5BMGGH_SYSTEM%5D_HP_1_DESIGN_NOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install CoolProp
from CoolProp.HumidAirProp import HAPropsSI
from CoolProp.CoolProp import PropsSI

     |████████████████████████████████| 4.0MB 2.8MB/s 


In [0]:
#[Dry Air and Water Properties]
class PROP():

  Enthalpy = 0.0        #[J/kg] 
  Desnity =0.0          #[kg/m^3]
  Viscosity = 0.0       #[Pa s]
  Heat_Capacity = 0.0   #[J/kg/K]
  Thermal_Cond = 0.0    #[W/m/K]

  def __init__(self,T,P,Fluid):
    self.Tdb = T+273.15
    self.P = P 
    self.Enthalpy = PropsSI("H", "P", P, "T", self.Tdb, Fluid)/1000
    self.Desnity = PropsSI("D", "P", P, "T", self.Tdb, Fluid)
    self.Viscosity = PropsSI("viscosity", "P", P, "T", self.Tdb, Fluid)*1000
    self.Heat_Capacity = PropsSI("Cpmass", "P", P, "T", self.Tdb, Fluid)/1000
    self.Thermal_Cond = PropsSI("conductivity", "P", P, "T", self.Tdb, Fluid)

  def Print_Prop(self):
    print('Enthalpy',self.Enthalpy, "kJ/kg")
    print('Desnity',self.Desnity, "kg/m^3")
    print("Viscosity",self.Viscosity, "mN-s/m2")
    print('Heat_Capacity',self.Heat_Capacity, "kJ/kg-C")
    print('Thermal_Cond',self.Thermal_Cond, "W/m-C")
    print()

In [0]:
#[Moist Air Properties]
class HAP():
     
    Enthalpy = 0.0
    Humidity_Ratio = 0.0                    #kg-water/kg-da
    Specific_Volume = 0.0                   #m3/kg
    Wet_Bulb_Temperature = 0.0
    Dew_Point_Temperature = 0.0
    Partial_pressure_of_water_vapor = 0.0
     
    def __init__(self,T,P,RH):
        self.Tdb = T+273.15
        self.P = P
        self.RH = RH

        self.Enthalpy = HAPropsSI('H','T',self.Tdb,'P',self.P,'R',self.RH)/1000
        self.Humidity_Ratio = HAPropsSI("W", "T", self.Tdb, "P", self.P, "R",self.RH)
        self.Specific_Volume = HAPropsSI("V", "T", self.Tdb, "P",self.P, "R",self.RH)
        self.Wet_Bulb_Temperature = HAPropsSI("Twb", "T", self.Tdb, "P", self.P, "R",self.RH)-273.15
        self.Dew_Point_Temperature = HAPropsSI("Tdp", "T", self.Tdb, "P", self.P, "R",self.RH)-273.15
        self.Partial_pressure_of_water_vapor = HAPropsSI("P_w", "T", self.Tdb, "P", self.P, "R",self.RH)

    def Print_Prop(self):
        print('Enthalpy',self.Enthalpy, "kJ/kg")
        print('Humidity_Ratio',self.Humidity_Ratio, "kg-water/kg-da")
        print("Specific_Volume",self.Specific_Volume, "m3/kg")
        print('Wet_Bulb_Temperature',self.Wet_Bulb_Temperature, "C")
        print('Dew_Point_Temperature',self.Dew_Point_Temperature, "C")
        print('Relative_Humidity',self.RH, "__")
        print()

In [0]:
#[Nm3/hr to kg/s]
def FRma(w,mafr):
  # w [kg/kg-da]
  # mafr [Nm3/hr]
  dwa=1.29227*(1/(1+w))+0.80378*(w/(1+w))
  return mafr* dwa /3600 # [kg/s]

# HP#1 原設計條件 取熱器

In [61]:
# 設計HP#1 取熱器水量 195 t/h 
  # WATER
  print("---------- Property As Water ----------")
  TW1 = 75
  TW2 = 115
  StateW1 = PROP(TW1,300000,"water")
  StateW2 = PROP(TW2,300000,"water")
  FR_w = 195*1000/3600
  print("FR_w", FR_w, "kg/s")
  print("Design Cooler Water Q",FR_w*abs(StateW1.Enthalpy-StateW2.Enthalpy),"kW")
  # NOTE DESIGN HEAT DUTY ON　WATER SIDE IS 9154 kJ/kg [OK]

# 設計HP#1 取熱器煙氣量 534600 Nm3/h 轉 kg/s 
  # DRY AIR
  print("---------- Property As Dry Air ----------")
  TA1 = 150
  TA2 = 105
  StateA1 = PROP(TA1,101325,"air")
  StateA2 = PROP(TA2,101325,"air")
  # Density of dry air(kg/Nm3): 1.29227
  FR_da = 534600*1.29227/3600
  print("FR_da", FR_da, "kg/s")
  print("Design Cooler Dry Air Q",FR_da*abs(StateA1.Enthalpy-StateA2.Enthalpy),"kW")

  # MOIST AIR
  # Assume Design Condition 8.4% (Water Volume/ Moist Air Volume)
  '''
  Molar mass of dry air = 28.965(kg/kmol)
  Molar mass of water = 18.016(kg/kmol)
  Density of moist air = (1.29227*91.6+0.80378*8.4)/100 [kg/Nm3]
  '''
  FR_ma = 534600*(1.29227*91.6+0.80378*8.4)/100/3600 
  print("FR_ma", FR_ma, "kg/s")

  




---------- Property As Water ----------
FR_w 54.166666666666664 kg/s
Design Cooler Water Q 9124.161163971838 kW
---------- Property As Dry Air ----------
FR_da 191.902095 kg/s
Design Cooler Dry Air Q 8759.006041444009 kW
FR_ma 185.80867074 kg/s


# HP#1 原設計條件 再熱器

In [57]:
# 設計HP#1 取熱器水量 195 t/h 
  # WATER
  print("---------- Property As Water ----------") 
  TW1 = 114.5
  TW2 = 80.9
  StateW1 = PROP(TW1,300000,"water")
  StateW2 = PROP(TW2,300000,"water")
  FR_w = 195*1000/3600
  print("FR_w", FR_w, "kg/s")
  Qw = FR_w*abs(StateW1.Enthalpy-StateW2.Enthalpy)
  print("Design Cooler Water Q",Qw,"kW")
  # NOTE DESIGN HEAT DUTY ON　WATER SIDE IS 7637.5 kJ/kg [OK]

# 設計HP#1 再熱器煙氣量 560046 Nm3/h 轉 kg/s
  # DRY AIR
  print("---------- Property As Dry Air ----------")
  TA1 = 55
  TA2 = 90
  StateA1 = PROP(TA1,101325,"air")
  StateA2 = PROP(TA2,101325,"air")
  # Density of dry air(kg/Nm3): 1.29227
  FR_da = 560046*1.29227/3600
  print("FR_da", FR_da, "kg/s")
  print("Design Cooler Dry Air Q",FR_da*abs(StateA1.Enthalpy-StateA2.Enthalpy),"kW")

  # MOIST AIR
  print("---------- Property As MOIST AIR @INLET RH = 100 ----------") 
  TMA1 = 55
  TMA2 = 90
  StateMA1 = HAP(TMA1,101325,1)
  StateMA2 = HAP(TMA2,101325,HAPropsSI('RH','T',TMA2+273.15,'P',101325,'W',StateMA1.Humidity_Ratio))
  FR_ma = FRma(StateMA1.Humidity_Ratio,560046)
  # Use dry air flow rate to calculate heat balanced
  FR_da = 560046/3600 * 1/(1+StateMA1.Humidity_Ratio)*1.29227
  print("StateMA2_RH", HAPropsSI('RH','T',TMA2+273.15,'P',101325,'W',StateMA1.Humidity_Ratio))
  print("FR_da", FR_da, "kg/s")
  print("Design Cooler Moist Air Q",FR_da*abs(StateMA1.Enthalpy-StateMA2.Enthalpy),"kW")

  # MOIST AIR
  print("---------- Property As MOIST AIR @INLET RH != 100 ----------") 
  TMA1 = 55
  TMA2 = 90
  Qma_ideal = Qw
  
  StateMA1_Up = 1
  StateMA1_Down =0.01
  StateMA1_Iterate = (StateMA1_Up+StateMA1_Down)/2

  while 1:
    StateMA1 = HAP(TMA1,101325,StateMA1_Iterate)
    StateMA2 = HAP(TMA2,101325,HAPropsSI('RH','T',TMA2+273.15,'P',101325,'W',StateMA1.Humidity_Ratio))
    FR_ma = FRma(StateMA1.Humidity_Ratio,560046)
    # Use dry air flow rate to calculate heat balanced
    FR_da = 560046/3600 * 1/(1+StateMA1.Humidity_Ratio)*1.29227
    
    Qma = FR_da*abs(StateMA1.Enthalpy-StateMA2.Enthalpy)
    
    if abs(Qma_ideal-Qma) <0.01:
      print("StateMA1_RH", StateMA1_Iterate)
      print("StateMA2_RH", HAPropsSI('RH','T',TMA2+273.15,'P',101325,'W',StateMA1.Humidity_Ratio))
      print("FR_da", FR_da, "kg/s")
      print("Design Cooler Moist Air Q",Qma,"kW")
      break
    elif (Qma_ideal-Qma) >0.01:
      StateMA1_Down = StateMA1_Iterate
      StateMA1_Iterate = (StateMA1_Up+StateMA1_Down)/2
    else:
      StateMA1_Up = StateMA1_Iterate
      StateMA1_Iterate = (StateMA1_Up+StateMA1_Down)/2


---------- Property As Water ----------
FR_w 54.166666666666664 kg/s
Design Cooler Water Q 7668.866532796981 kW
---------- Property As Dry Air ----------
FR_da 201.03629011666666 kg/s
Design Cooler Dry Air Q 7099.062902781411 kW
---------- Property As MOIST AIR @INLET RH = 100 ----------
StateMA2_RH 0.22492332464755546
FR_da 180.24887338522117 kg/s
Design Cooler Moist Air Q 7768.710074859196 kW
---------- Property As MOIST AIR @INLET RH != 100 ----------
StateMA1_RH 0.8642556762695311
StateMA2_RH 0.19439126005206436
FR_da 183.22263329591598 kg/s
Design Cooler Moist Air Q 7668.866370065676 kW


# 操作HP#1 循環水量為 149 m3/h，轉換成 ton/h 

In [3]:
StateW = PROP(85.6,300000,"water")
StateW.Print_Prop()
149.0 * 968.3117055144205 /1000

Enthalpy 358.72510770610853 kJ/kg
Desnity 968.3117055144205 kg/m^3
Viscosity 0.33075732660653967 mN-s/m2
Heat_Capacity 4.200815017052788 kJ/kg-C
Thermal_Cond 0.6705206520130683 W/m-C



144.27844412164865

# 操作HP#1，取熱器(水側)熱平衡 (kW)

In [6]:
StateW1 = PROP(85.6,300000,"water")
StateW2 = PROP(117.7,300000,"water")

print(144.3*1000/3600*abs(StateW1.Enthalpy-StateW2.Enthalpy))

5427.390400130433


# 操作HP#1，再熱器(水側)熱平衡 (kW)

In [7]:
StateW1 = PROP(85.4,300000,"water")
StateW2 = PROP(117.5,300000,"water")

print(144.3*1000/3600*abs(StateW1.Enthalpy-StateW2.Enthalpy))

5427.0809579748875
